# Convert a Keras model to TensorFlow.js format

In [1]:
sequence_length = 30  # Number of frames per sequence
shape_no_face = (sequence_length, 258)
num_classes = 4
actions = ['hallo','du','idle','a']  # List all your actions including idle
DATA_PATH = 'CollectionBase/MP_Data/30frame_no_face_+idle'  # Update with your data path
landmarks = [False,True,True,True] # Select True/False for different landmarks (face, pose, left hand, right hand)

In [2]:
import h5py
import tensorflow as tf

# Load your model
model = tf.keras.models.load_model('ai_L20_d02.h5')

# Print the summary to inspect the layers and weights
model.summary()

# Optionally, rename duplicate weights
for layer in model.layers:
    if hasattr(layer, 'weights'):
        for i, weight in enumerate(layer.weights):
            if 'gamma' in weight.name:
                new_name = f"{weight.name.split(':')[0]}_{i}"
                weight._handle_name = new_name

# Save the model again after renaming weights
model.save('ai_L20_d02.h5')


2024-06-13 08:53:54.203507: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 08:53:54.286931: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 08:53:54.561055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 08:53:56.029763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-13 08:53:57.287882: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/b

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_10 (Masking)            │ (None, 30, 258)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_27 (LSTM)                  │ (None, 30, 128)        │       198,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 30, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_28 (LSTM)                  │ (None, 30, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 30, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_29 (LSTM)                  │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 496,902 (1.90 MB)

 Trainable params: 496,132 (1.89 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 2 (12.00 B)

In [3]:

import numpy as np
from keras.src.layers import Masking
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import os
import tensorflowjs as tfjs

l2_factor = 0.00
dropout = 0.2

model = Sequential()

# Input layer
model.add(Masking(mask_value=0., input_shape=shape_no_face))  # Adjust input shape accordingly

model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(LSTM(128, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))

model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_factor)))

model.load_weights('ai_L20_d02.h5')



tfjs.converters.save_keras_model(model, './output_directory')

/home/eelam/Desktop/AiSL/AiSignLanguage/tf_env/lib/python3.12/site-packages/tensorflowjs/read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations